In [1]:
import math
import numpy as np
import pandas as pd
from fun_utils import *
from scipy.optimize import minimize, least_squares
from scipy.optimize import fsolve
from scipy.integrate import quad
import matplotlib.pyplot as plt

In [2]:
## scenario switch
case=3;  # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0; # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [3]:
## parameter values
alpha = 0.85;           # labor share parameter in manufacturing
theta = 4;              # scopevec for comparative advantage
sigma = 1;      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1;  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma;  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar;  #Foreign's elasticity of demand for embodied energy
# beta = 2.274853;
# gamma= 0.784877595;
beta=1.892412;
gamma=0.807998928;
epsilonS = 0.5;  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5;  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [4]:
ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)

In [5]:
df = pd.read_csv("../output/BaselineCarbon_2015.csv", index_col=['region_scenario','regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])
df=df.drop([1,2,4,5,6,7])  
df

,,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
region_scenario,regionbase,,,,,,,,,,,,,,,
3,OECD37 as Home,8.625495,23.650503,32.276001,11.293668,2.487537,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498


In [ ]:
np.append(np.arange(0,35,0.1), (np.arange(35,200,2)))

In [ ]:
np.append((np.append(np.arange(0,35,0.1), (np.arange(35,200,2)))), np.arange(200,1000,5))

In [6]:
tax_scenario= pd.DataFrame({'tax_sce': 'Unilateral', 'Base':0},index=[1])

In [56]:
def iterate_varphi(tax_scenario, df, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    # varphilist = np.arange (1.6,2.5,0.1) # marginal damages
    varphilist = np.append((np.append(np.arange(0,35,0.1), (np.arange(35,200,2)))), np.arange(200,1000,5)) # marginal damages

    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    #varphilist = [2]
    output=[]
    
    # initial guess for optimizers, for subsequent optimizations the initial guess will be the previous optimal value
    prevtb = 0
    prevpe = 1
    
    for varphi in varphilist:
        # nested minimizer with sum of absolute value difference as objective
        #print(df)        
        tax_df = df.apply(assign_eq, axis = 1, args = (prevtb, prevpe, varphi, ParaList, tax_scenario))
        prevtb = tax_df['tb']
        prevpe = tax_df['pe']
        #print(tax_df)
        
        appended_df = pd.merge(df, tax_df, on=['region_scenario','regionbase'])

        ## back out other optimal values
        output_df = appended_df.apply(compute_all, axis=1, raw=False, result_type=None, args=(varphi, tax_scenario, ParaList))
        output_df = pd.merge(tax_df, output_df, on=['region_scenario','regionbase'])
        output.append(output_df)
        #print(varphi)

    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb

    print(tax_scenario['tax_sce'])
    return output


In [66]:
def objective(p, varphi, tax_scenario, ParaList, df):
    pe = p[0]
    tb_mat = p[1:]
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList

    #solve for equilibrium
    if tax_scenario['tax_sce']!='Unilateral':
        print("shouldn't be here")
        return 0
        
     ## optimal values
    # jxbar_hat =   (1 - df['jxbar']) ** (-1) / (((1 - df['jxbar']) ** (-1) - 1) + (1 + (1 - alpha) * tb_mat[0]/pe) ** (-theta) * (1 + tb_mat[0]/pe) ** ((1 - alpha) * theta));
    jxbar_hat = pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * tb_mat[0])**(-theta));
    j0_hat = (pe+tb_mat[0])**(-(1-alpha)*theta) / (df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta))
    jmbar_hat = 1
    
    if tax_scenario['tax_sce']=='Unilateral':
        te=varphi;

    
    jxbar_prime = jxbar_hat * df['jxbar'];
    jmbar_prime = jmbar_hat * df['jmbar'];
    
    j0_prime = j0_hat * df['jxbar'];

    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1)) 
    
    incomp_beta1 = quad(tempFunction,0,j0_prime, args=(theta, sigmastar))
    incomp_beta2 = quad(tempFunction,0,jxbar_prime, args=(theta, sigmastar))
    Bfunvec1_prime = incomp_beta1[0]
    Bfunvec2_prime = incomp_beta2[0]
    if (incomp_beta1[1] > 0.01 or incomp_beta2[1] > 0.01):
        print('integral convergence failed')

    #if te is too large, Home stop producing
    petbte = pe + tb_mat[0] - te;
    z = pe + tb_mat[0] >= te;
    petbte = petbte * z;

    Qe_hat = (petbte) ** epsilonS;
    Qestar_hat = pe ** epsilonSstar;
          
    if logit==1:
        epsilonS=beta*(1-gamma)/(1-gamma+gamma*petbte**beta);
        epsilonSstar=beta*(1-gamma)/(1-gamma+gamma*pe**beta);
        Qe_hat = (petbte)**beta/(1-gamma+gamma*(petbte)**beta);
        Qestar_hat = pe**beta/(1-gamma+gamma*pe**beta);
 
    Qe_prime = df['Qe'] * Qe_hat;
    Qestar_prime = df['Qestar'] * Qestar_hat;
     
    CeHH_hat = (pe + tb_mat[0]) ** (-epsilonD) * jmbar_hat ** (1 + (1 - sigma)/theta);
    CeHH_prime = df['CeHH'] * CeHH_hat;
       
    
    # CeFH_hat = (1 + (1 - sigmastar)/theta) * pe ** (-(1 - alpha) * sigmastar) * (pe + tb_mat[0]) ** (-alpha) * Bfunvec_prime/(df['jxbar'] ** (1 +1/theta)) * (1 - df['jxbar']) ** (sigmastar/theta);
    CeFH1_hat = (pe +tb_mat[0])**(-epsilonDstar) * j0_hat**(1 + (1 - sigmastar)/theta);
    CeFH2_hat = (1 + (1 - sigmastar)/theta) * ((1-df['jxbar'])/df['jxbar'])**(sigmastar/theta) * pe**(-epsilonDstar) * (1 + tb_mat[0]/pe)**(-alpha) * (Bfunvec2_prime - Bfunvec1_prime)/df['jxbar']**(1+(1-sigmastar)/theta);
    CeFH1_prime = df['CeFH'] * CeFH1_hat;
    CeFH2_prime = df['CeFH'] * CeFH2_hat;
    CeFH_hat = CeFH1_hat + CeFH2_hat;
    
    if np.isnan(CeFH_hat)==True:
        CeFH_hat=0
    CeFH_prime =df['CeFH'] * CeFH_hat;
    
    
    CeHF_hat = (pe + tb_mat[0]) ** (-epsilonD);
    CeHF_prime = df['CeHF'] * CeHF_hat;
    
    
    CeFF_prime = df['CeFF'] * ((1 - jxbar_prime)/(1-df['jxbar'])) ** (1 + (1 - sigmastar)/theta) * pe ** (-epsilonDstar)
    
    ##
    VgHH = df['CeHH']/(1 - alpha);
    VgFF = df['CeFF']/(1 - alpha);
    
    VgFH = df['CeFH'] /(1 - alpha);
    # VgFH_prime = VgFH * pe ** ((1 - sigmastar) * (1 - alpha)) * (1 - (1 - jxbar_prime) ** (1 + (1 - sigmastar)/theta))/ (df['jxbar'] * (1 - df['jxbar']) ** ( (1-sigmastar)/theta));
    VgFH1_hat = (pe + tb_mat[0]) * CeFH1_hat;
    VgFH2_hat = pe**(1 - epsilonDstar) * ((1-j0_prime)**(1+(1-sigmastar)/theta) - (1-jxbar_prime)**(1+(1-sigmastar)/theta))/ (df['jxbar']  * (1 - df['jxbar'] )**( (1-sigmastar)/theta));
    VgFH1_prime = VgFH * VgFH1_hat;
    VgFH2_prime = VgFH * VgFH2_hat;
    
    
    diff = Qe_prime + Qestar_prime - (CeHH_prime + CeFH_prime + CeHF_prime + CeFF_prime);
    pai_g = (pe + tb_mat[0]) * CeFH2_prime / (1 - alpha) - VgFH2_prime
    
    numerator = varphi * epsilonSstar * Qestar_prime - sigmastar * (1-alpha) * pai_g
    denominator = epsilonSstar * Qestar_prime + epsilonDstar * CeFF_prime
    diff1 = tb_mat[0] * denominator - numerator
        
    return abs(diff) + abs(diff1)

def assign_eq(df, prevtb, prevpe, varphi, ParaList, tax_scenario):
    #print(df)
    #print(df['regionbase'])
    res1 = minimize(objective, (prevpe, prevtb), args = (varphi, tax_scenario, ParaList, df), bounds = [(0,np.inf), (0, np.inf)], method = 'Nelder-Mead', options = {'maxfev': 100000})
    diff = verify([res1.x[1]], res1.x[0], varphi, tax_scenario, ParaList, df)
    prevpe = res1.x[0]
    prevtb = res1.x[1]
        
    # technically we should grid search here for a new start, but experiment shows that starting from 0 works so...
    if (abs(diff[0]) > 0.01) or (abs(diff[1]) > 0.01):
        res1 = minimize(objective, (0, 1), args = (varphi, tax_scenario, ParaList, df), bounds = [(0,np.inf), (0, np.inf)], method = 'Nelder-Mead', options = {'maxfev': 100000})
        diff1 = verify([res1.x[1]], res1.x[0], varphi, tax_scenario, ParaList, df)
        prevpe = res1.x[0]
        prevtb = res1.x[1]
        # manual simple multistart
        if (abs(diff1[0]) > 0.01) or (abs(diff1[1]) > 0.01):
            print('diff too big', diff, diff1, varphi)
    return pd.Series({'pe': prevpe, 'tb': prevtb})

In [67]:
output_all=tax_scenario.apply(iterate_varphi, axis=1, args=(df, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))

C:\Users\Alex\AppData\Local\Temp/ipykernel_6580/386902465.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  jxbar_hat = pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * tb_mat[0])**(-theta));
C:\Users\Alex\AppData\Local\Temp/ipykernel_6580/386902465.py:13: RuntimeWarning: invalid value encountered in double_scalars
  jxbar_hat = pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * tb_mat[0])**(-theta));
C:\Users\Alex\AppData\Local\Temp/ipykernel_6580/386902465.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  j0_hat = (pe+tb_mat[0])**(-(1-alpha)*theta) / (df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta))
C:\Users\Alex\AppData\Local\Temp/ipykernel_6580/386902465.py:59: Runtim

diff too big (3.3569305379899106e-07, -0.7114257929622738) (-2.8064731516508346e-07, -0.7114259440264732) 27.200000000000003
diff too big (3.188585397495558e-06, -1.4641424067055766) (1.8266402141572726, -1.693358797183464e-06) 27.3
Unilateral


In [59]:
output_list=[]
for i in range(1,len(tax_scenario)+1):
    output_list.append(output_all.loc[i])
outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
outcomes.reset_index(level=0, inplace=True)

In [60]:
outcomes

,tax_sce,region_scenario,pe,tb,varphi,jxbar_prime,jmbar_prime,j0_prime,Qe_prime,Qestar_prime,...,leakage2,leakage3,chg_extraction,chg_production,chg_consumption,chg_Qeworld,pai_g,subsidy_ratio,welfare,welfare_noexternality
regionbase,,,,,,,,,,,,,,,,,,,,,
OECD37 as Home,Unilateral,3,1.000000,0.000000,0.0,0.049234,0.819498,0.049234,8.625495,23.650503,...,0.033333,0.323333,0.000000,1.000000e-07,9.700000e-07,-0.000003,0.000000,0.000000,-4.317474e-07,-4.317474e-07
OECD37 as Home,Unilateral,3,0.993667,0.039981,0.1,0.049253,0.819498,0.048138,8.334422,23.575491,...,0.084004,0.225309,-0.075012,3.075279e-02,8.248284e-02,-0.366088,-0.204258,0.005999,1.992536e-02,-1.992110e-02
OECD37 as Home,Unilateral,3,0.988369,0.079572,0.2,0.049307,0.819498,0.047105,8.035798,23.512558,...,0.065028,0.204957,-0.137945,4.731719e-02,1.491358e-01,-0.727645,-0.412435,0.011932,7.957643e-02,-7.882309e-02
OECD37 as Home,Unilateral,3,0.984046,0.118887,0.3,0.049394,0.819498,0.046128,7.729005,23.461087,...,0.045979,0.184435,-0.189416,4.992940e-02,2.002801e-01,-1.085909,-0.624856,0.017800,1.782903e-01,-1.762933e-01
OECD37 as Home,Unilateral,3,0.980653,0.158018,0.4,0.049511,0.819498,0.045204,7.413262,23.420600,...,0.026879,0.163754,-0.229903,3.876288e-02,2.361562e-01,-1.442139,-0.841802,0.023600,3.158152e-01,-3.120568e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OECD37 as Home,Unilateral,3,0.001272,350.504976,975.0,1.000000,0.819498,0.000028,0.000000,0.843625,...,-0.638344,-0.562811,-22.806878,-2.006466e+01,-1.769049e+01,-31.432376,-2121.686932,0.999976,3.178852e+04,-1.568393e+03
OECD37 as Home,Unilateral,3,0.001264,352.256496,980.0,1.000000,0.819498,0.000028,0.000000,0.840827,...,-0.638288,-0.562844,-22.809676,-2.006469e+01,-1.769309e+01,-31.435174,-2132.319529,0.999976,3.195959e+04,-1.571370e+03
OECD37 as Home,Unilateral,3,0.001256,354.007875,985.0,1.000000,0.819498,0.000028,0.000000,0.838053,...,-0.638233,-0.562876,-22.812450,-2.006473e+01,-1.769567e+01,-31.437948,-2142.951271,0.999976,3.213068e+04,-1.574336e+03


In [61]:
outcomes.to_csv('../output/convergent.csv'.format(case), header=True) 